https://mlflow.org/docs/latest/deep-learning/tensorflow/guide/index.html

In [1]:
import mlflow
import numpy as np
import tensorflow as tf
from tensorflow import keras

2024-06-26 14:05:18.581032: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


logging system metrics

In [2]:
import os

os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"

defining tracking server

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("my-experiment")
mlflow.enable_system_metrics_logging()

logging training

In [4]:
mlflow.tensorflow.autolog()

# Prepare data for a 2-class classification.
data = np.random.uniform(size=[20, 28, 28, 3])
label = np.random.randint(2, size=20)

model = keras.Sequential(
    [
        keras.Input([28, 28, 3]),
        keras.layers.Conv2D(8, 2),
        keras.layers.MaxPool2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(2),
        keras.layers.Softmax(),
    ]
)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

with mlflow.start_run() as run:
    model.fit(data, label, batch_size=5, epochs=5)

2024-06-26 14:05:21.004846: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024/06/26 14:05:21 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/06/26 14:05:21 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/5
4/4 [==============================] - ETA: 0s - loss: 0.7109 - sparse_categorical_accuracy: 0.5000

2024/06/26 14:05:22 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


4/4 [==============================] - 1s 43ms/step - loss: 0.7109 - sparse_categorical_accuracy: 0.5000
Epoch 2/5
4/4 [==============================] - ETA: 0s - loss: 0.6286 - sparse_categorical_accuracy: 0.6000

2024/06/26 14:05:22 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


...

4/4 [==============================] - 0s 33ms/step - loss: 0.5009 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
4/4 [==============================] - ETA: 0s - loss: 0.5038 - sparse_categorical_accuracy: 0.9500

2024/06/26 14:05:23 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


1/1 [==============================] - 0s 131ms/step


INFO:tensorflow:Assets written to: /tmp/tmpswtyptxk/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpswtyptxk/model/data/model/assets
2024/06/26 14:05:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/thiagoads/Workspace/Programas/miniconda3/envs/tensorflow-mlflow/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
2024/06/26 14:05:31 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/26 14:05:31 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


loading logged model

In [5]:
logged_model = 'runs:/{}/model'.format(run.info.run_id)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model.predict(data)

1/1 [==============================] - 0s 62ms/step


array([[0.6427531 , 0.3572469 ],
       [0.3066249 , 0.6933752 ],
       [0.37581766, 0.62418234],
       [0.6403687 , 0.35963127],
       [0.6500231 , 0.34997684],
       [0.39852977, 0.60147023],
       [0.33723497, 0.662765  ],
       [0.6236628 , 0.37633714],
       [0.3485793 , 0.6514207 ],
       [0.29357612, 0.7064238 ],
       [0.35833946, 0.6416606 ],
       [0.6474158 , 0.3525842 ],
       [0.7299636 , 0.2700364 ],
       [0.47465557, 0.52534443],
       [0.649153  , 0.35084707],
       [0.43805364, 0.56194633],
       [0.4384147 , 0.56158537],
       [0.6726473 , 0.32735267],
       [0.39639416, 0.6036058 ],
       [0.32622686, 0.6737731 ]], dtype=float32)